# Project: When was the Golden Age of Video Games
## Creating a persistent duckDB database

### Assumptions:
-  [DuckDB](https://duckdb.org/docs/guides/python/jupyter) and [JupySQL](https://jupysql.ploomber.io/en/latest/quick-start.html) are installed.
> %pip install jupysql duckdb-engine --quiet   
> You may need to restart the kernel to use updated packages.

### Caveats:
- `video_games.duckdb` was created using DuckDB version 0.9.2. See [here](https://duckdb.org/internals/storage) for related info.
  

The following are steps to create a persistent DudkDB database for use with the project [notebook](https://github.com/Zirescu/when_was_the_golden_age_of_video_games/blob/main/when_was_the_golden_age_of_video_games.ipynb). While it isn't a requirement to use a persistent database I've chosen to use one to show the flexibility of DuckDB. 


### 1. Load in the sql extension and setup a connection to a persistent DuckDB database.

In [1]:
%load_ext sql
%sql duckdb:///video_games.duckdb --alias video_games

Connecting to 'video_games'

### 2. Execute the queries to create the tables and load in our data from the corresponding CSV files.

In [3]:
%%sql 
DROP TABLE IF EXISTS game_sales;

CREATE TABLE game_sales (
  game VARCHAR(100) PRIMARY KEY,
  platform VARCHAR(64),
  publisher VARCHAR(64),
  developer VARCHAR(64),
  games_sold NUMERIC(5, 2),
  year INT
);

DROP TABLE IF EXISTS reviews;

CREATE TABLE reviews (
    game VARCHAR(100) PRIMARY KEY,
    critic_score NUMERIC(4, 2),   
    user_score NUMERIC(4, 2)
);

DROP TABLE IF EXISTS top_critic_years;

CREATE TABLE top_critic_years (
    year INT PRIMARY KEY,
    avg_critic_score NUMERIC(4, 2)  
);

DROP TABLE IF EXISTS top_critic_years_more_than_four_games;

CREATE TABLE top_critic_years_more_than_four_games (
    year INT PRIMARY KEY,
    num_games INT,
    avg_critic_score NUMERIC(4, 2)  
);

DROP TABLE IF EXISTS  top_user_years_more_than_four_games;

CREATE TABLE top_user_years_more_than_four_games (
    year INT PRIMARY KEY,
    num_games INT,
    avg_user_score NUMERIC(4, 2)  
);

COPY game_sales FROM 'csv/game_sales.csv' (AUTO_DETECT TRUE);
COPY reviews FROM 'csv/game_reviews.csv' (AUTO_DETECT TRUE);
COPY top_critic_years FROM 'csv/top_critic_scores.csv' (AUTO_DETECT TRUE);
COPY top_critic_years_more_than_four_games FROM 'csv/top_critic_scores_more_than_four_games.csv' (AUTO_DETECT TRUE);
COPY top_user_years_more_than_four_games FROM 'csv/top_user_scores_more_than_four_games.csv' (AUTO_DETECT TRUE);

Running query in 'video_games'

Count


### 3. Validate our tables and columns were created using the [duckdb_tables](https://duckdb.org/docs/sql/duckdb_table_functions#duckdb_tables) function

In [18]:
%%sql 
SELECT 
    database_name,
    table_name,
    has_primary_key,
    estimated_size,
    column_count
FROM duckdb_tables();


Running query in 'video_games'

database_name,table_name,has_primary_key,estimated_size,column_count
video_games,game_sales,True,400,6
video_games,reviews,True,400,3
video_games,top_critic_years,True,10,2
video_games,top_critic_years_more_than_four_games,True,10,3
video_games,top_user_years_more_than_four_games,True,10,3


### 4. Show and then close our database connection

In [29]:
%sql -l


current,url,alias
*,duckdb:///video_games.duckdb,video_games


In [30]:
%sql --close video_games

In [31]:
%sql -l

current,url,alias


**This concludes creating the database.**

### Note
I've noticed with VSCodium that if this notebook remains open it still maintians some hold over the database file so you may need to close out the notebook or restart the kernel. 

**Sample error message:**
> RuntimeError: (duckdb.duckdb.IOException) IO Error: Cannot open file "...": The process cannot access the file because it is being used by another process.
